In [ ]:
pip install tensorboardX

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import torchvision.transforms as T
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

import tensorboardX

In [ ]:
# 모델 정의하기

class LeNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.s2 = nn.AvgPool2d(2,2)
        self.c3  = nn.Conv2d(in_channels= 6, out_channels= 16 , kernel_size=5)
        self.s4 = nn.AvgPool2d(2,2)
        self.c5 = nn.Conv2d(in_channels= 16, out_channels= 120, kernel_size=5)


        self.classifier = nn.Sequential(
             nn.Linear(in_features= 120, out_features=84),
            nn.Tanh(), # 원래 FC layer에서도 tanh를 적용해줘야하나?
            nn.Linear(in_features=84, out_features=10)
        )

        # 여기서 어떻게 84 -> 10개로 줄엿지?

    def forward(self,x):
        # 입력 데이터는 28

        # C1에 입력 데이터 넘겨주기
        # C1에서 나온 데이터에 weight를 곱하고, bias를 더해준다. -> 이는 nn.Conv2d 내부에서 적용되어서 결과가 나온다
        # tanh 함수 적용하기
        result = F.tanh(self.c1(x))
        result = self.s2(result)

        # pooling 레이어에 Dropout 적용 후 C3 입력으로 넣어줌
       # result = F.dropout(result,p=0.3) # p : default value = 0.5 , 0.3
        result = F.tanh(self.c3(result))

        result = self.s4(result)

        result = self.c5(result)
        result = torch.flatten(result,1)

        logits = self.classifier(result)

        probs = F.softmax(logits,dim=1) # softmax를 통해 결과를 확률값으로 변환해준다.

        return logits, probs

In [ ]:
# ToTensor()를 통해 PIL Image를 FloatTensor로 변경하고 이미지 값들이 0과 1사이에 위치하도록 조정한다.
# size를 32 x 32로 변경한다.
# 입력 픽셀 값들은 background는 -0.1로 foreground는 1.175로 정규화된다

transform = T.Compose([
            T.ToTensor(),
            T.Resize(32),
            T.Normalize(mean=0, std=1)
             ]
        )

train_dataset = datasets.MNIST(
    root='./data', # 데이터셋을 저장할 경로
    train=True,
    download=True,
    transform= transform
)

test_dataset = datasets.MNIST(
    root='./data',
    train = False,
    download=True,
    transform=transform
)

#image , label = train_dataset[0]

#print(image.shape) # 원본 : torch.Size([1, 28, 28]) -> resize 를 통해 torch.Size([1, 32, 32])로 수정
#label # 5

#plt.imshow(image.squeeze(0),cmap='gray')

In [ ]:
# 모델에 데이터를 넘겨주기 위해 Dataset과 DataLoader를 구성해보자 .
# MNIST 데이터 자체가 Dataset이므로 우리는 DataLoader만 구성하면 된다.

train_dataloader = DataLoader(train_dataset,batch_size = 128,shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = 128, shuffle= True)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorboard
import datetime

# Set the log directory
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Start TensorBoard within the notebook
%tensorboard --logdir logs/fit

In [ ]:
from tensorboardX import SummaryWriter
writer = SummaryWriter(log_dir)

In [ ]:
def show_loss(loss,epoch):
    plt.plot(epoch, loss)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

In [ ]:
def train_model(model, data_loader):

    running_loss = 0
    epoch_loss = 0

    for data, target in data_loader:

        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        # 순전파
        logits, prediction = model(data)

        # Removed .to(torch.float32) as CrossEntropyLoss expects long type
        loss = criterion(logits, target.to(torch.long) )
        running_loss += loss.item() * target.size(0)
        # 역전파
        loss.backward() # 기울기 누적
        optimizer.step() # weight, bias 업데이트


    epoch_loss = running_loss / len(data_loader.dataset)
    return model, epoch_loss

In [ ]:
# 정확도 평가 함수
def get_accuracy(model, data_loader):

    n = 0
    true_label = 0
    model.eval()
    with torch.no_grad():
        for data, target in data_loader:
            data = data.to(device)
            target = target.to(device)

            logits ,probs = lenet(data)

            probs = probs.squeeze() # 1차원 제거해주기

            max_prob, label = torch.max(probs,1)

            n += target.size(0)
            true_label += (target == label).sum() # 맞은 예측의 개수
        accuracy = true_label.float() / n
    return accuracy


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

lenet = LeNet()

lenet = lenet.to(device)

# nn.MSELoss() 대신  CrossEntropyLoss 사용
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lenet.parameters(), lr=0.0005)


# 모델을 학습 시켜보자

epochs = 5

train_losses = []
test_losses = []

lenet.train()
for epoch in range(epochs):
    lenet ,loss = train_model(lenet, train_dataloader)
    show_loss(loss, epoch)
    accuracy = get_accuracy(lenet, train_dataloader)

    print(f'epoch {epoch} loss : {loss}')
    # print(f'epoch {epoch} loss : {np.mean(train_losses)}')
    writer.add_scalar("Loss/train", loss, epoch)

In [ ]:
writer.flush()

In [ ]:
%tensorboard --logdir logs/fit

### RuntimeError: mat1 and mat2 shapes cannot be multiplied (7680x1 and 120x84)
- 7680 : 64(배치 크기) * 120 (입력 크기)
- 120 * 84 : 입력 크기, 출력 크기

In [ ]:
print("Model's state_dict:")
for param_tensor in lenet.state_dict():
    print(param_tensor, "\t", lenet.state_dict()[param_tensor].size())

#torch.save(lenet, './model')

In [ ]:
#lenet = torch.load('model',weights_only=True)
lenet.eval()
with torch.no_grad():
    accuracy = get_accuracy(lenet, test_dataloader) * 100
print(f'Accuracy: {accuracy:.2f}%')
